
## Operations


### Addition

Given two points in a set governed by an elliptic curve with a point at infinity (?)

$$
E\ =\ \{(x,y) | y^2 = x^3 + ax + b \} \cup \{ O \}
$$

We can write $$P + Q$$ such that we have a third point $$-R$$ that can be reflected to gve us $$R$$. 

This resuts in the following algebra for slope and point location

$$ s = \frac{y_P - y_Q}{x_P - x_Q} $$
$$ x_R = s^2 - (x_P + x_Q) $$
$$ y_R = s(x_p - x_R) - y_P $$

(need a picture?)

#### Exception \#1: Point Doubling

$$ P + P = R = 2P $$

This becomes a special case algebraically as its basically the tangential point

$$ s = \frac{3x_P^2 + a}{2y_P} $$
$$ x_R = s^2 - 2x_P $$
$$ y_R = s(x_P - x_R) - y_P $$


#### Exception \#2: Adding Vertical Points

What if Q & P share their own x value? Then we have the point at infinity $$O$$

Put another way: $$ P + Q = O $$ if $$ x_P = x_Q $$ or $$ x_P = 0 $$



### Scalar Multiplication

Given a point _P_ within the group defined by the elliptic curve _E_, we can compute a scalar multiplication as a repeated addition: 

$$ Q = kP $$

where _k_ is any real, positive integer. 

$$ P \in E $$
$$ k \in \mathbb{Z} $$
$$ Q = kP $$

Becomes 

$$ Q = P + P + ... + P $$ 

Over k times


## One Way Functions

What we want is a one way function. This will allow us to create a public key from any private key but prevent a private key from being created from a public key. 

Finding _k_, our private key, such that $ Q = kP $ is all but impossible with an elliptic curve that is parameterized correctly. 

## The Process

First we generate a cyclic group such that 

$$ G \in E (\mathbb{Z} / p \mathbb{Z} ) $$

And the size of this subgroup:

$$ ord(G) = n $$

Which corresponds to our point at infinity:

$$ kG = O $$

And a cofactor $h$:

$$ h = \frac{|E(\mathbb{Z}/p\mathbb{Z}|}{n} $$ 

where $h$ is ideally $h=1$.


### The Shared Parameters

$$ \{p, a, b, G, n, h \} $$

$p$: field modulo -- typically a very large prime

$a,b$: curve parameters for $E$

$G$: our generator point

$n$: size of the subgroup

$h$: cofactor


## A Tale of Alice & Bob

Alice and Bob agree on these basic parameters seen above. They then exchange public keys. These public keys are then multiple against their private keys and they can have the same encryption cipher (or whatever mechanism they want to use)

$$ A = aG $$

$$ B = bG $$

For Bob: $ Ba = (bG)a = abG $

For Alive: $ Aa = (aG)b = abG $

Which is identical(!) -- if Eve intercepts this, she can't do much because, although she has $A$, $B$, and $G$, it's impossible to find the private keys as:

$$ B = bG $$

$$ \frac{B}{G} = b $$

cannot be feasibly computed. 

## Working Example

$$ E: y^2 \equiv X^3 + 2x + 2 $$

where $ p = 17 $

and our generator point is $ G = (5,1) $

We then compute the cyclic group by continuinally multipling $G$ until we get to $0,0$ (I think?)

That leaves our $ n = 19 $ -- values will repeat after $19G$ such that $20G = G$ (hence cyclic group)

You can then pick random numbers for the private keys, generate the public keys, etc. -- I think these private keys need to be under $n$ but I could be wrong.

# Different Working Example Using the Ed25519 Equation

$$
E: y^2 \equiv x^3 + 486662x^2 + x
$$

$ n = 2^{252} + 27742317777372353535851937790883648493 $

and 

$ p = 2^{255} - 19 $


What would be a good generator point?

In [83]:
prime

57896044618658097711785492504343953926634992332820282019728792003956564819949

In [119]:
elliptic_multiply(2, (6,3))

(6, 3)


KeyboardInterrupt: 

In [166]:
from typing import Tuple
import random

def multiplicative_inverse(x: int, modulo: int) -> int:
    '''
    Generate the multiplicative inverse using the extended
    eucliean algorithm -- for modulo divisions
    
    Parameters x & modulo come from => x % modulo
    
    >>> multiplicative_inverse(3, 17)
    6
    >>> multiplicative_inverse(2, 17)
    9
    '''
    a = 1
    left_hand_mult = x * a
    right_hand_mod = 1 % modulo
    while (left_hand_mult % modulo) != right_hand_mod:
        a += 1
        left_hand_mult = x * a
    return a

def ed25519(x: int, prime: int = ((2**255) - 19)) -> int:
    '''
    Returns the squared y value in an ed25519 equation:
    
    y^2 = x^3 + 486662x^2 + x
    
    where a = 486662 and b = 1
    
    :param x: value to pass to the curve
    :param prime: prime value to use for the modulo operation, 
        defaults to (2**255) - 19
    
    :returns: y^2 in the above equation
    
    >>> ed25519(5)
    12166680
    '''
    return ((x**3) + (486662 * (x**2)) + x) % prime

def elliptic_add(P: Tuple[int, int], 
                 Q: Tuple[int, int],
                 a: int = 486662,
                 prime: int = ((2**255) - 19)) -> Tuple[int, int]:
    '''
    Given two points P & Q, return the third point R using 
    elliptic addition. Includes exceptions for point doubling
    (P == Q) and infinity points (P & Q share the same x)
    
    :param P: point P
    :param Q: point Q
    :param a: a parameter in the elliptic curve, defaults to 286662
        for the ed25519 curve
    :param prime: the prime number for modulo operations, defaults
        to (2**255) - 19 from the ed25519 curve
    
    :returns: point P + Q
    
    >>> elliptic_add(P=(5,1), G=(5,1), a=2, prime=_prime)
    (6, 3)
    
    >>> elliptic_add(P=(7,6), G=(5,1), a=2, prime=_prime)
    (7, 11)
    '''
    x_p, y_p = P
    x_q, y_q = Q    
    
    # point doubling
    if P == Q:
        s_num = 3 * (x_p**2) + a
        s_denom = 2 * y_p
        s = (s_num % prime) * multiplicative_inverse(s_denom, prime) 
        x_r = (s**2) - (2 * x_p)
        y_r = (s*(x_p - x_r)) - y_p
    
    # adding vertical points -- infinite number allowed(?)
    elif x_p == x_q:
        return (float('inf'), float('inf'))
    
    # standard addition
    else:
        s_num = (y_p - y_q)
        s_denom = (x_p - x_q)
        s = (s_num % prime) * multiplicative_inverse(s_denom, prime)
        x_r = (s**2) - (x_p + x_q)
        y_r = ( s * (x_p - x_r) ) - y_p
    
    return (int(x_r % prime), int(y_r % prime))
    

def elliptic_multiply(x: int, 
                      G: Tuple[int, int],
                      a: int = 486662,
                      prime: int = ((2**255) - 19)) -> Tuple[int, int]:
    '''
    Multiply a point G by a integer scalar a using elliptic
    arithmetic. 
    
    :param x: scalar value
    :param G: the point value
    :param a: equivalent parameter from the curve, defaults to 486662 
        from the ed25519 curve
    :param prime: prime number to use, defaults to (2**255) - 19 
        from the ed25519 curve
    
    :returns: new point
    
    >>> elliptic_multiply(x=10, G=(5,1), a=2, prime=17)
    (7, 11)
    
    >>> elliptic_multiply(x=19, G=(5,1), a=2, prime=_prime)
    (inf, inf)
    
    >>> elliptic_multiply(x=21, G=(5,1), a=2, prime=_prime)
    (6, 3)
    '''
    running_G = G
    for i in range(x-1):
        if running_G == (float('inf'), float('inf')):
            running_G = G
            continue 
        running_G = elliptic_add(running_G, G, a, prime)
    return running_G

In [175]:
# Example from the video
_prime = 17
a, b= 2, 2;   # curve parameters
_curve = lambda x: int(((x**3) + (a*x) + b) % _prime)
norder = 19  # could be computed by trying to multiply until we get infinity
G = (5, 1)

bob_private_key = 9
bob_public_key = elliptic_multiply(bob_private_key, G, a, _prime)
alice_private_key = 3
alice_public_key = elliptic_multiply(alice_private_key, G, a, _prime)

print(bob_private_key, bob_public_key)
print(alice_private_key, alice_public_key)

alice_cipher = elliptic_multiply(alice_private_key, bob_public_key, a, _prime)
bob_cipher = elliptic_multiply(bob_private_key, alice_public_key, a, _prime)

print("Same Cipher? %r => %s" % (alice_cipher == bob_cipher, alice_cipher))

9 (7, 6)
3 (10, 6)
Same Cipher? True => (13, 7)


In [221]:
"0b%s" % make_binary("My secret message in binary")

'0b010011010111100100100000011100110110010101100011011100100110010101110100001000000110110101100101011100110111001101100001011001110110010100100000011010010110111000100000011000100110100101101110011000010111001001111001'

In [226]:
# you'd probably join these two for the cipher
def make_binary(s):
    return "".join([format(ord(c), '08b') for c in s])

message = bin("0b%s" % make_binary("My secret message in binary"))
key = bin(0b100)
encrypted = message ^ key
print(message, encrypted)

TypeError: 'str' object cannot be interpreted as an integer

Sources
  
  * [This YT video](https://www.youtube.com/watch?v=F3zzNa42-tQ)
  * [Math Examples](http://www.herongyang.com/EC-Cryptography/Algebraic-Point-Addition-Example.html) -- notably includes details on how R can be verified as belonging to Q & P
  * [Finding the multiplicative inverse of modulo operations](https://www.youtube.com/watch?v=Gu7iKt2SZYc)